<a href="https://colab.research.google.com/github/abarb2022/Walmart-Recruiting---Store-Sales-Forecasting/blob/main/model_experiment_arima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Downloading Kaggle data sets directly into Colab**

Install the kaggle python library

In [ ]:
! pip install kaggle

Mount the Google drive so you can store your kaggle API credentials for future use

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Make a directory for kaggle at the temporary instance location on Colab drive.

Download your kaggle API key (.json file). You can do this by going to your kaggle account page and clicking 'Create new API token' under the API section.

In [5]:
! mkdir ~/.kaggle

Upload the json file to Google Drive and then copy to the temporary location.

In [6]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

Change the file permissions to read/write to the owner only

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

**Competitions and Datasets are the two types of Kaggle data**

**1. Download competition data**

If you get 403 Forbidden error, you need to click 'Late Submission' on the Kaggle page for that competition.

In [8]:
! kaggle competitions download -c walmart-recruiting-store-sales-forecasting

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 1.12GB/s]


Unzip, in case the downloaded file is zipped. Refresh the files on the left hand side to update the view.

In [9]:
! unzip walmart-recruiting-store-sales-forecasting

Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [10]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder # For Type encoding if not using category dtype directly
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import gc # For garbage collection
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [11]:
stores = pd.read_csv('stores.csv')
train = pd.read_csv("train.csv.zip")
features = pd.read_csv('features.csv.zip')
sample = pd.read_csv('sampleSubmission.csv.zip')
test = pd.read_csv('test.csv.zip')

In [12]:
# Convert 'Date' columns to datetime objects for easier manipulation
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])
features['Date'] = pd.to_datetime(features['Date'])

# Merge features with train and test data.
# Note: 'IsHoliday' is present in both train/test and features.csv.
# We'll merge on it to ensure consistency, but if there were discrepancies,
# we'd need a more careful merge strategy.
train_df = pd.merge(train, features, on=['Store', 'Date', 'IsHoliday'], how='left')
test_df = pd.merge(test, features, on=['Store', 'Date', 'IsHoliday'], how='left')

# Merge store information
train_df = pd.merge(train_df, stores, on='Store', how='left')
test_df = pd.merge(test_df, stores, on='Store', how='left')

print("\n--- Merged Train Data Head ---")
print(train_df.head())
print("\n--- Merged Test Data Head ---")
print(test_df.head())

print("\n--- Merged Train Data Info ---")
print(train_df.info())
print("\n--- Merged Test Data Info ---")
print(test_df.info())

# Free up memory
del train, test, features, stores
gc.collect()


--- Merged Train Data Head ---
   Store  Dept       Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment Type    Size
0      1     1 2010-02-05      24924.50      False        42.31       2.572        NaN        NaN        NaN        NaN        NaN  211.096358         8.106    A  151315
1      1     1 2010-02-12      46039.49       True        38.51       2.548        NaN        NaN        NaN        NaN        NaN  211.242170         8.106    A  151315
2      1     1 2010-02-19      41595.55      False        39.93       2.514        NaN        NaN        NaN        NaN        NaN  211.289143         8.106    A  151315
3      1     1 2010-02-26      19403.54      False        46.63       2.561        NaN        NaN        NaN        NaN        NaN  211.319643         8.106    A  151315
4      1     1 2010-03-05      21827.90      False        46.50       2.625        NaN        NaN        NaN        Na

0

## **DATA CLEANING**


In [18]:
class MissingValueImputer(BaseEstimator, TransformerMixin):
    """
    Custom Transformer to handle missing values for specific columns.
    - MarkDown columns: fill with 0.
    - Other specified numerical columns: fill with ffill then bfill, fallback to mean.
    """
    def __init__(self, markdown_cols=None, numerical_cols_to_impute=None):
        self.markdown_cols = markdown_cols if markdown_cols is not None else [f'MarkDown{i}' for i in range(1, 6)]
        self.numerical_cols_to_impute = numerical_cols_to_impute if numerical_cols_to_impute is not None else ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
        self.means = {} # To store means for fallback imputation during transform

    def fit(self, X, y=None):
        # Calculate means for fallback imputation from the training data
        for col in self.numerical_cols_to_impute:
            if col in X.columns:
                self.means[col] = X[col].mean()
        return self

    def transform(self, X):
        X_copy = X.copy()


        for col in self.markdown_cols:
          if col in X_copy.columns:
            X_copy[f"{col}_was_missing"] = X_copy[col].isna().astype(int)
            X_copy[col] = X_copy[col].fillna(0)


        # Impute other numerical columns with ffill then bfill, fallback to mean
        for col in self.numerical_cols_to_impute:
            if col in X_copy.columns:
                X_copy[col] = X_copy[col].ffill().bfill()
                # Fallback to mean if NaNs still exist (e.g., if all values were NaN in a column)
                if X_copy[col].isnull().any() and col in self.means:
                    X_copy[col] = X_copy[col].fillna(self.means[col])
        return X_copy

In [14]:
class DateFeatureExtractor(BaseEstimator, TransformerMixin):
    """
    Custom Transformer to extract temporal features from the 'Date' column.
    """
    def __init__(self, date_column='Date'):
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        if self.date_column not in X_copy.columns:
            raise ValueError(f"Date column '{self.date_column}' not found in DataFrame.")

        X_copy[self.date_column] = pd.to_datetime(X_copy[self.date_column])

        X_copy['Year'] = X_copy[self.date_column].dt.year
        X_copy['Month'] = X_copy[self.date_column].dt.month
        X_copy['Month_sin'] = np.sin(2 * np.pi * X_copy['Month'] / 12)
        X_copy['Month_cos'] = np.cos(2 * np.pi * X_copy['Month'] / 12)

        # Using .dt.isocalendar().week for consistent week numbering across years
        X_copy['Week'] = X_copy[self.date_column].dt.isocalendar().week.astype(int)
        X_copy['Day'] = X_copy[self.date_column].dt.day
        X_copy['DayOfWeek'] = X_copy[self.date_column].dt.dayofweek

        X_copy['Week_sin'] = np.sin(2 * np.pi * X_copy['Week'] / 52)
        X_copy['Week_cos'] = np.cos(2 * np.pi * X_copy['Week'] / 52)

        # Markdown aggregation
        X_copy['Total_MarkDown'] = X_copy[[f'MarkDown{i}' for i in range(1, 6)]].sum(axis=1)
        X_copy['MarkDown_Intensity'] = X_copy['Total_MarkDown'] / (X_copy['Total_MarkDown'].mean() + 1)

        # Economic indicators
        X_copy['Fuel_CPI_Ratio'] = X_copy['Fuel_Price'] / X_copy['CPI']
        X_copy['Economic_Index'] = (X_copy['CPI'] * 0.4 + (100 - X_copy['Unemployment']) * 0.6) / 100


        # Convert IsHoliday to integer if it exists and is boolean
        if 'IsHoliday' in X_copy.columns and X_copy['IsHoliday'].dtype == bool:
            X_copy['IsHoliday'] = X_copy['IsHoliday'].astype(int)

        # Keep the 'Date' column for ARIMA
        return X_copy # Removed .drop(columns=[self.date_column, "Month", "Week"])

In [15]:
y_train = train_df['Weekly_Sales']
X_train = train_df.drop(columns=['Weekly_Sales', 'Id'], errors='ignore')

temp_train_df = X_train.copy()
temp_train_df['Date'] = pd.to_datetime(train_df['Date']) # Get original dates back for sorting
temp_train_df['Weekly_Sales'] = y_train

temp_train_df = temp_train_df.sort_values(by='Date').reset_index(drop=True)

# Define a cutoff date for validation
validation_cutoff_date = pd.to_datetime('2012-09-01')

X_train_split = temp_train_df[temp_train_df['Date'] < validation_cutoff_date]
y_train_split = temp_train_df[temp_train_df['Date'] < validation_cutoff_date]['Weekly_Sales']

X_val_split = temp_train_df[temp_train_df['Date'] >= validation_cutoff_date]
y_val_split = temp_train_df[temp_train_df['Date'] >= validation_cutoff_date]['Weekly_Sales']

def weighted_mean_absolute_error(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

val_weights = np.where(X_val_split['IsHoliday'] == 1, 5, 1)


In [21]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
from statsmodels.tools.sm_exceptions import ConvergenceWarning, ValueWarning
from joblib import Parallel, delayed


class ARIMAModelWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, order, seasonal_order=(0,0,0,0), verbose=True):
        self.order = order
        self.seasonal_order = seasonal_order
        self.verbose = verbose
        self.models = {}
        self.last_values = {}
        self.global_average = None  # <- NEW


    def fit(self, X, y=None):
        # Suppress all statsmodels warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=ConvergenceWarning)
            warnings.simplefilter("ignore", category=UserWarning)
            warnings.simplefilter("ignore", category=ValueWarning)

            grouped = X.groupby(['Store', 'Dept'])

            # Create progress bar if verbose
            if self.verbose:
                groups = tqdm(grouped, desc="Training ARIMA models", unit="store-dept")
            else:
                groups = grouped

            if 'Weekly_Sales' in X.columns:
                self.global_average = X['Weekly_Sales'].mean()


            for (store, dept), group in groups:
                ts_data = group.set_index('Date')['Weekly_Sales']

                # Force weekly frequency to prevent warnings
                ts_data = ts_data.asfreq('W-FRI')

                if len(ts_data.dropna()) < 3:
                  if self.verbose:
                      print(f"Skipping Store {store}, Dept {dept} due to insufficient data")
                  continue


                if len(ts_data) > 0:
                    try:
                        with warnings.catch_warnings():
                            warnings.simplefilter("ignore")
                            model = ARIMA(ts_data,
                                         order=self.order,
                                         seasonal_order=self.seasonal_order)
                            fitted_model = model.fit()
                            self.models[(store, dept)] = fitted_model
                            self.last_values[(store, dept)] = ts_data.iloc[-1]
                    except Exception as e:
                        if self.verbose:
                          print(f"Failed on Store {store}, Dept {dept} - {str(e)}")



                        continue
        return self

    def transform(self, X):
        # This will return ARIMA predictions for the existing dates
        # For production, you might want a separate predict method
        return X

    def predict(self, X):
        # Make predictions for each store-dept combination in X
        predictions = []
        for _, row in X.iterrows():
            store = row['Store']
            dept = row['Dept']
            date = row['Date']

            if (store, dept) in self.models and self.models[(store, dept)] is not None:
                try:
                     # Get the forecast for this specific date
                    model = self.models[(store, dept)]
                    # Calculate the number of steps from the last training data point to the prediction date
                    # Assuming weekly data frequency
                    # Find the last date the model was trained on
                    last_train_date = model.model.data.dates[-1]
                    steps = (date - last_train_date).days // 7


                    if steps >= 0: # Predict from the last training date onwards
                        forecast = model.forecast(steps=steps + 1) # Forecast up to the prediction date
                        pred = forecast.iloc[-1]
                    else: # If the date is before the last training date, use the observed value
                         # This case should ideally not happen in a standard forecast scenario,
                         # but included for robustness if predict is used on past dates.
                         # We would need to find the closest date in the training data
                        print(f"Warning: Predicting for a date before the last training date for Store {store}, Dept {dept}, Date {date}")
                        if (store, dept) in self.last_values:
                          pred = self.last_values[(store, dept)]
                        else:
                          # Safe fallback value (mean of all sales?)
                          print("Safe fallback value (mean of all sales?)")
                          pred = global_average


                except Exception as e:
                    print(f"Prediction failed for Store {store}, Dept {dept}, Date {date}: {str(e)}")
                    pred = self.last_values[(store, dept)] if (store, dept) in self.last_values else np.nan # Fallback to last value or NaN
            else:
                # Fallback - use last known value or NaN if no model was fitted
                pred = self.last_values[(store, dept)] if (store, dept) in self.last_values else np.nan


            predictions.append(pred)

        return np.array(predictions)

In [36]:
# Define the full pipeline
arima_order = (1,0,1)  # Simplified order

arima_seasonal_order=(0,0,0,0)
# Preprocessing steps
preprocessing = Pipeline([
    ('missing_value_imputer', MissingValueImputer()),
    ('date_feature_extractor', DateFeatureExtractor())
    ])



# Full pipeline with ARIMA
full_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('arima_model', ARIMAModelWrapper(order=arima_order, seasonal_order=arima_seasonal_order))
])


full_pipeline.fit(X_train_split, y_train_split)
predictions = full_pipeline.predict(X_val_split)






/tmp/ipython-input-12-1027183038.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')
Training ARIMA models:   4%|▍         | 139/3326 [00:08<03:40, 14.47store-dept/s]

Skipping Store 2, Dept 77 due to insufficient data


Training ARIMA models:   6%|▋         | 214/3326 [00:14<02:42, 19.17store-dept/s]

Skipping Store 3, Dept 78 due to insufficient data


Training ARIMA models:   7%|▋         | 218/3326 [00:14<02:46, 18.62store-dept/s]

...

Training ARIMA models:  98%|█████████▊| 3255/3326 [03:53<00:05, 14.18store-dept/s]

Skipping Store 44, Dept 99 due to insufficient data


Training ARIMA models: 100%|█████████▉| 3323/3326 [03:59<00:00, 14.15store-dept/s]

Skipping Store 45, Dept 96 due to insufficient data


Training ARIMA models: 100%|██████████| 3326/3326 [04:00<00:00, 13.86store-dept/s]
/tmp/ipython-input-12-1027183038.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [37]:
def weighted_mean_absolute_error(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

val_weights = np.where(X_val_split['IsHoliday'] == 1, 5, 1)
print (weighted_mean_absolute_error(y_val_split, predictions, val_weights))


1999.510169856433


In [25]:
!pip install dagshub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 7.5 MB/s eta 0:00:00


In [26]:
!pip install mlflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.4/741.4 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 19.2 MB/s eta 0:00:00


In [27]:

import dagshub, mlflow
# Try to get credentials from environment first
dagshub.init(
    repo_owner='abarb22',
    repo_name='Walmart-Recruiting---Store-Sales-Forecasting',
    mlflow=True
)
mlflow.set_experiment("ARIMA_Training")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d97ecdc7-59f0-4ba1-92d9-48b871aa37a1&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a86998497631a0d85891aae820edd2f8a3535f6a5d13671b86bc05af844acbfc




Accessing as abarb22

Initialized MLflow to track repo "abarb22/Walmart-Recruiting---Store-Sales-Forecasting"

Repository abarb22/Walmart-Recruiting---Store-Sales-Forecasting initialized!

<Experiment: artifact_location='mlflow-artifacts:/6e01db02c8e240aebfa89d3184cdf829', creation_time=1751573162919, experiment_id='2', last_update_time=1751573162919, lifecycle_stage='active', name='ARIMA_Training', tags={}>

In [ ]:
with mlflow.start_run(run_name="ARIMA_Data_Cleaning"):
    # Log data cleaning parameters
    mlflow.log_param("missing_value_strategy", "MarkDowns->0, others->ffill/bfill/mean")
    mlflow.log_param("date_features_extracted", True)


    # Log metrics about data quality
    mlflow.log_metric("cleaned_missing_values", train_df.isna().sum().sum())


In [ ]:
with mlflow.start_run(run_name="ARIMA_Feature_Engineering"):
    # Log feature engineering parameters
    mlflow.log_params({
        "temporal_features": ["Year", "Month", "Week", "DayOfWeek"],
        "cyclical_features": ["Month_sin", "Month_cos", "Week_sin", "Week_cos"],
        "economic_features": ["Fuel_CPI_Ratio", "Economic_Index"],
        "markdown_features": ["Total_MarkDown", "MarkDown_Intensity"]
    })

    # Your feature engineering
    feature_pipeline = Pipeline([
        ('date_extractor', DateFeatureExtractor())
    ])

    X_featured = feature_pipeline.fit_transform(X_train_split)

    # Log results
    mlflow.log_metric("total_features", len(X_featured.columns))
    mlflow.log_metric("time_span_days", (X_featured['Date'].max() - X_featured['Date'].min()).days)

In [ ]:


with mlflow.start_run(run_name="ARIMA_Model_Training"):
    # Log model parameters
    arima_params = {
        'order': (1,0,1),
        'seasonal_order': (0,0,0,0),
        'trend': 'c'
    }
    mlflow.log_params(arima_params)

    preprocessing = Pipeline([
        ('missing_value_imputer', MissingValueImputer()),
        ('date_feature_extractor', DateFeatureExtractor()),
    ])

    # Full pipeline with ARIMA
    full_pipeline = Pipeline([
        ('preprocessing', preprocessing),
        ('arima_model', ARIMAModelWrapper(order=arima_params['order'], seasonal_order=arima_params['seasonal_order']))
    ])


    full_pipeline.fit(X_train_split, y_train_split)
    val_preds = full_pipeline.predict(X_val_split)


    val_wmae = weighted_mean_absolute_error(y_val_split, val_preds, val_weights)
        # Log metrics
    mlflow.log_metrics({
        "train_samples": len(X_train_split),
        "val_samples": len(X_val_split),
        "val_wmae": val_wmae,
    })

    # Log model (as artifact since statsmodels doesn't have native MLflow support)
    import joblib
    joblib.dump(full_pipeline, "arima_pipeline.joblib")
    mlflow.log_artifact("arima_pipeline.joblib")


/tmp/ipython-input-11-1027183038.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')
/tmp/ipython-input-17-489002183.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = X.groupby(['Store', 'Dept'])
/tmp/ipython-input-11-1027183038.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such

In [41]:
with mlflow.start_run(run_name="ARIMA_Model_Training"):
    # Log model parameters
    arima_params = {
        'order': (1, 0, 1),
        'seasonal_order': (0, 0, 0, 0),
        'trend': 'c'
    }
    mlflow.log_params(arima_params)

    preprocessing = Pipeline([
        ('missing_value_imputer', MissingValueImputer()),
        ('date_feature_extractor', DateFeatureExtractor()),
    ])

    arima_model = ARIMAModelWrapper(order=arima_params['order'], seasonal_order=arima_params['seasonal_order'], verbose=True)

    # Full pipeline with ARIMA
    full_pipeline = Pipeline([
        ('preprocessing', preprocessing),
        ('arima_model', arima_model)
    ])

    full_pipeline.fit(X_train_split, y_train_split)
    val_preds = full_pipeline.predict(X_val_split)

    val_wmae = weighted_mean_absolute_error(y_val_split, val_preds, val_weights)

    # Extra info from your model
    trained_models_count = len(arima_model.models)
    fallback_preds_count = np.sum(pd.Series(val_preds).isna() | pd.Series(val_preds) == arima_model.global_average)
    skipped_count = (X_train_split.groupby(['Store', 'Dept']).ngroups) - trained_models_count

    # Log metrics
    mlflow.log_metrics({
        "train_samples": len(X_train_split),
        "val_samples": len(X_val_split),
        "val_wmae": val_wmae,
        "trained_groups": trained_models_count,
        "skipped_groups": skipped_count,
        "fallback_predictions": int(fallback_preds_count),
    })

    if arima_model.global_average is not None:
        mlflow.log_metric("global_average_fallback_value", arima_model.global_average)

    # Log model artifact
    import joblib
    joblib.dump(full_pipeline, "arima_pipeline.joblib")
    mlflow.log_artifact("arima_pipeline.joblib")


/tmp/ipython-input-12-1027183038.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')
Training ARIMA models:   4%|▍         | 136/3326 [00:28<25:41,  2.07store-dept/s]

Skipping Store 2, Dept 77 due to insufficient data


Training ARIMA models:   6%|▋         | 213/3326 [00:49<04:42, 11.03store-dept/s]

Skipping Store 3, Dept 78 due to insufficient data


Training ARIMA models:   7%|▋         | 218/3326 [00:49<04:54, 10.56store-dept/s]

...

Skipping Store 44, Dept 99 due to insufficient data


Training ARIMA models: 100%|██████████| 3326/3326 [04:56<00:00, 11.23store-dept/s]


Skipping Store 45, Dept 96 due to insufficient data


/tmp/ipython-input-12-1027183038.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


🏃 View run ARIMA_Model_Training at: https://dagshub.com/abarb22/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2/runs/89b439f8294e4e10a4ce360e057cd16d
🧪 View experiment at: https://dagshub.com/abarb22/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2


In [28]:
from itertools import product
import mlflow
import joblib

param_grid = list(product([0, 1, 2], [0, 1], [0, 1, 2]))  # (p, d, q)

preprocessing = Pipeline([
    ('missing_value_imputer', MissingValueImputer()),
    ('date_feature_extractor', DateFeatureExtractor())
])

best_order = None
best_wmae = float('inf')

with mlflow.start_run(run_name="ARIMA_Model_GridSearch"):
    for order in param_grid:
        print(f"Trying ARIMA order: {order}")
        arima_model = ARIMAModelWrapper(order=order, seasonal_order=(0,0,0,0), verbose=False)

        pipeline = Pipeline([
            ('preprocessing', preprocessing),
            ('arima_model', arima_model)
        ])

        try:
            # Start a nested run for this specific ARIMA(p,d,q) config
            with mlflow.start_run(run_name=f"ARIMA_{order}", nested=True):
                pipeline.fit(X_train_split, y_train_split)
                preds = pipeline.predict(X_val_split)
                val_wmae = weighted_mean_absolute_error(y_val_split, preds, val_weights)

                print(f"WMAE for order {order}: {val_wmae:.4f}")

                # Log parameters and metrics for this sub-run
                mlflow.log_params({
                    'order_p': order[0],
                    'order_d': order[1],
                    'order_q': order[2],
                })
                mlflow.log_metric("val_wmae", val_wmae)

                if val_wmae < best_wmae:
                    best_wmae = val_wmae
                    best_order = order

        except Exception as e:
            print(f"Order {order} failed: {e}")
            continue

    print(f"\n✅ Best ARIMA order found: {best_order} with WMAE: {best_wmae:.4f}")

    # Optional: train and log best model
    final_pipeline = Pipeline([
        ('preprocessing', preprocessing),
        ('arima_model', ARIMAModelWrapper(order=best_order, seasonal_order=(0,0,0,0)))
    ])

    final_pipeline.fit(X_train_split, y_train_split)
    final_preds = final_pipeline.predict(X_val_split)
    final_wmae = weighted_mean_absolute_error(y_val_split, final_preds, val_weights)

    mlflow.log_params({
        'best_order_p': best_order[0],
        'best_order_d': best_order[1],
        'best_order_q': best_order[2],
        'seasonal_order': (0, 0, 0, 0)
    })

    mlflow.log_metrics({
        "train_samples": len(X_train_split),
        "val_samples": len(X_val_split),
        "best_val_wmae": final_wmae
    })

    joblib.dump(final_pipeline, "best_arima_pipeline.joblib")
    mlflow.log_artifact("best_arima_pipeline.joblib")


Trying ARIMA order: (0, 0, 0)
WMAE for order (0, 0, 0): 2425.4089
🏃 View run ARIMA_(0, 0, 0) at: https://dagshub.com/abarb22/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2/runs/81fa95ac04ad41ed8422f195a098c80c
🧪 View experiment at: https://dagshub.com/abarb22/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2
Trying ARIMA order: (0, 0, 1)
WMAE for order (0, 0, 1): 2424.8848
🏃 View run ARIMA_(0, 0, 1) at: https://dagshub.com/abarb22/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2/runs/2acc72532e8949c5abd697975e5e25b8
🧪 View experiment at: https://dagshub.com/abarb22/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2
Trying ARIMA order: (0, 0, 2)
WMAE for order (0, 0, 2): 2298.0891
🏃 View run ARIMA_(0, 0, 2) at: https://dagshub.com/abarb22/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2/runs/07caeed2bb5c4f988851f050229f421e
🧪 View experiment at: https://dagshub.com/abarb22/Walmart-Recruiting---

Training ARIMA models:   4%|▍         | 138/3326 [00:12<04:01, 13.23store-dept/s]

Skipping Store 2, Dept 77 due to insufficient data


Training ARIMA models:   6%|▋         | 213/3326 [00:20<04:26, 11.70store-dept/s]

Skipping Store 3, Dept 78 due to insufficient data


...

Skipping Store 44, Dept 99 due to insufficient data


Training ARIMA models: 100%|█████████▉| 3325/3326 [05:53<00:00,  9.75store-dept/s]

Skipping Store 45, Dept 96 due to insufficient data


Training ARIMA models: 100%|██████████| 3326/3326 [05:53<00:00,  9.41store-dept/s]


🏃 View run ARIMA_Model_GridSearch at: https://dagshub.com/abarb22/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2/runs/1e212ae6ec984521b6dbb3e28069426f
🧪 View experiment at: https://dagshub.com/abarb22/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/2
